In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential

2024-03-31 00:06:27.495990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## This is a weather forecasting model using LSTM. I will use the data of 3 days before the chosen date to predict it's max tempeture

In [5]:
data = pd.read_csv('./downloads/weather.csv')
data.head(10)

,province,max,min,wind,wind_d,rain,humidi,cloud,pressure,date
0,Bac Lieu,27,22,17,NNE,6.9,90,71,1010,2009-01-01
1,Bac Lieu,31,25,20,ENE,0.0,64,24,1010,2010-01-01
2,Bac Lieu,29,24,14,E,0.0,75,45,1008,2011-01-01
3,Bac Lieu,30,24,30,E,0.0,79,52,1012,2012-01-01
4,Bac Lieu,31,25,20,ENE,0.0,70,24,1010,2013-01-01
5,Bac Lieu,28,23,14,ENE,0.0,75,55,1012,2014-01-01
6,Bac Lieu,29,23,10,ENE,0.4,75,42,1012,2015-01-01
7,Bac Lieu,32,24,22,ENE,0.0,63,9,1015,2016-01-01
8,Bac Lieu,30,24,20,ENE,0.5,76,35,1011,2017-01-01
9,Bac Lieu,29,23,16,E,0.0,70,33,1010,2018-01-01


In [8]:
# sort the data by date and province
df_sorted = data.sort_values(by=['province', 'date'])
df_sorted.head(10)

,province,max,min,wind,wind_d,rain,humidi,cloud,pressure,date
0,Bac Lieu,27,22,17,NNE,6.9,90,71,1010,2009-01-01
117,Bac Lieu,28,22,15,ENE,0.5,85,61,1010,2009-01-02
91,Bac Lieu,23,21,9,ESE,16.7,91,77,1011,2009-01-03
104,Bac Lieu,27,21,9,E,2.2,86,32,1011,2009-01-04
39,Bac Lieu,29,22,10,SE,0.0,81,25,1010,2009-01-05
65,Bac Lieu,30,23,10,E,0.3,81,25,1011,2009-01-06
78,Bac Lieu,29,24,12,ESE,1.9,83,30,1012,2009-01-07
26,Bac Lieu,29,22,10,NE,1.2,81,40,1013,2009-01-08
13,Bac Lieu,28,21,12,ENE,0.0,74,26,1014,2009-01-09
40,Bac Lieu,26,18,14,E,0.0,69,16,1014,2009-01-10


In [15]:
df = df_sorted.copy()
df.index = [df['province'], df['date']]

required_cols = ['max', 'min', 'rain', 'humidi', 'cloud', 'pressure']
df = df[required_cols]

# drop the province and date columns
# df = df.drop(columns=['province', 'date'])

df.head(10)

max  min  rain  humidi  cloud  pressure
province date                                               
Bac Lieu 2009-01-01   27   22   6.9      90     71      1010
         2009-01-02   28   22   0.5      85     61      1010
         2009-01-03   23   21  16.7      91     77      1011
         2009-01-04   27   21   2.2      86     32      1011
         2009-01-05   29   22   0.0      81     25      1010
         2009-01-06   30   23   0.3      81     25      1011
         2009-01-07   29   24   1.9      83     30      1012
         2009-01-08   29   22   1.2      81     40      1013
         2009-01-09   28   21   0.0      74     26      1014
         2009-01-10   26   18   0.0      69     16      1014

In [16]:
df.isna().sum()

max         0
min         0
rain        0
humidi      0
cloud       0
pressure    0
dtype: int64

In [23]:
df

max  min  rain  humidi  cloud  pressure
province date                                               
Bac Lieu 2009-01-01   27   22   6.9      90     71      1010
         2009-01-02   28   22   0.5      85     61      1010
         2009-01-03   23   21  16.7      91     77      1011
         2009-01-04   27   21   2.2      86     32      1011
         2009-01-05   29   22   0.0      81     25      1010
...                  ...  ...   ...     ...    ...       ...
Yen Bai  2021-06-14   32   23   7.3      79     71      1001
         2021-06-15   34   25   0.3      71     46       999
         2021-06-16   36   27   0.0      64     29      1000
         2021-06-17   37   28   0.0      66     27      1002
         2021-06-18   37   28   0.0      65     19      1002

[181960 rows x 6 columns]

In [20]:
num_features = len(df.columns)
num_features

6

In [26]:
from sklearn.preprocessing import MinMaxScaler

# min max scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)


def normalize(df):
    result = df.copy()
    data_mean = df.mean(axis=0)
    data_std = df.std(axis=0)
    result = (df - data_mean) / data_std
    return result
# scale_df = normalize(scale_df)

len(scaled_data)

181960

In [30]:
scaled_data[1]

array([0.57142857, 0.66666667, 0.00083836, 0.80519481, 0.61      ,
       0.44      ])

In [54]:
# Create sequences
sequence_length = 10

sequences = []
labels = []

for i in range(len(scaled_data)-sequence_length):
    sequences.append(scaled_data[i:i+sequence_length])
    labels.append(scaled_data[i+sequence_length][0]) # next max temperature to be y_hat

In [55]:
print(sequences[0])
print(labels[0])

[[5.47619048e-01 6.66666667e-01 1.15694165e-02 8.70129870e-01
  7.10000000e-01 4.40000000e-01]
 [5.71428571e-01 6.66666667e-01 8.38363514e-04 8.05194805e-01
  6.10000000e-01 4.40000000e-01]
 [4.52380952e-01 6.33333333e-01 2.80013414e-02 8.83116883e-01
  7.70000000e-01 4.60000000e-01]
 [5.47619048e-01 6.33333333e-01 3.68879946e-03 8.18181818e-01
  3.20000000e-01 4.60000000e-01]
 [5.95238095e-01 6.66666667e-01 0.00000000e+00 7.53246753e-01
  2.50000000e-01 4.40000000e-01]
 [6.19047619e-01 7.00000000e-01 5.03018109e-04 7.53246753e-01
  2.50000000e-01 4.60000000e-01]
 [5.95238095e-01 7.33333333e-01 3.18578135e-03 7.79220779e-01
  3.00000000e-01 4.80000000e-01]
 [5.95238095e-01 6.66666667e-01 2.01207243e-03 7.53246753e-01
  4.00000000e-01 5.00000000e-01]
 [5.71428571e-01 6.33333333e-01 0.00000000e+00 6.62337662e-01
  2.60000000e-01 5.20000000e-01]
 [5.23809524e-01 5.33333333e-01 0.00000000e+00 5.97402597e-01
  1.60000000e-01 5.20000000e-01]]
0.5238095238095238


In [56]:
# Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)
print(sequences.shape)
print(labels.shape)

(181950, 10, 6)
(181950,)


In [57]:
# train test split
train_size = int(0.8 * len(sequences))

# split the data
x_train, y_train = sequences[:train_size], labels[:train_size]
x_test, y_test = sequences[train_size:], labels[train_size:]

print("Train X shape:", x_train.shape)
print("Train Y shape:", y_train.shape)
print("Test X shape:", x_test.shape)
print("Test Y shape:", y_test.shape)

Train X shape: (145560, 10, 6)
Train Y shape: (145560,)
Test X shape: (36390, 10, 6)
Test Y shape: (36390,)


# Build model

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Create model LSTM model
model = Sequential()

# Add LSTM layer with dropout layer
model.add(LSTM(units=128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

/Users/user/miniconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 10, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 10, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130,977 (511.63 KB)

 Trainable params: 130,977 (511.63 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
4094/4094 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - loss: 0.0087 - val_loss: 0.0030
Epoch 2/100
1441/4094 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0024 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming you have trained the model and have the 'best_model' object
# Also, 'test_x' and 'test_y' should be available

# Predict temperatures using the trained model
predictions = model.predict(test_x)

# Calculate evaluation metrics
mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)